# Prepare session

In [1]:
import os
os.chdir("..")
!pwd

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.local import LocalSession
import s3fs
import subprocess
from sagemaker.s3 import S3Downloader, S3Uploader
from pathlib import Path
import json

image_name = "sagemaker-t5"
ecr_namespace = image_name + "/"
default_bucket = "t5-engine-bucket"
default_uri = "s3://" + default_bucket
atf_s3_uri = default_uri + "/sagemaker"

role = get_execution_role()
account_id = role.split(":")[4]
boto_session = boto3.Session()
region = boto_session.region_name
bucket = default_bucket

sagemaker_session = sagemaker.Session(
    boto_session=boto_session,
    default_bucket=default_bucket
)
    
s3_helper = s3fs.S3FileSystem()
data_location_uri = default_uri + "/training_data/dummy"

print(account_id)
print(region)
print(role)
print(sagemaker_session)
print(default_uri)
print(atf_s3_uri)
print(data_location_uri)

852039983533
ap-southeast-1
arn:aws:iam::852039983533:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
s3://t5-engine-bucket
s3://t5-engine-bucket/sagemaker
s3://t5-engine-bucket/training_data/dummy


# Dev in real

## Build and push image

In [4]:
! cd container && bash build_image.sh $image_name Dockerfile

Sending build context to Docker daemon   47.1kB
Step 1/17 : ARG DEBIAN_FRONTEND=noninteractive
Step 2/17 : FROM python:3.8-slim
 ---> 05394ddb89cd
Step 3/17 : MAINTAINER Tung Dao <tungdao17@gmail.com>
 ---> Using cache
 ---> 7be6e8991878
Step 4/17 : RUN apt-get update
 ---> Using cache
 ---> 95c5c5339ecb
Step 5/17 : RUN apt-get install -y --no-install-recommends     wget     nginx     ca-certificates     apt-utils     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> c644696d398c
Step 6/17 : RUN apt-get update
 ---> Using cache
 ---> 5d0f1b4e8902
Step 7/17 : RUN apt-get -y install curl
 ---> Using cache
 ---> 0444cecfea93
Step 8/17 : RUN apt-get -y install libgomp1
 ---> Using cache
 ---> 3027dc4b1b01
Step 9/17 : COPY code/requirements.txt /opt/program/requirements.txt
 ---> Using cache
 ---> 005e51ca7352
Step 10/17 : RUN pip install -r /opt/program/requirements.txt
 ---> Using cache
 ---> e8d5a6cb2c30
Step 11/17 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 0911c54a8dbd
Ste

In [5]:
! cd container && bash push_image.sh $image_name

852039983533.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-t5:latest
Login
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [852039983533.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-t5]

56cd6421: Preparing 
2dbb8c69: Preparing 
c3efb15d: Preparing 
59e633e5: Preparing 
619f759b: Preparing 
711a9638: Preparing 
33d607b7: Preparing 
6f3a4937: Preparing 
b4c52667: Preparing 
450cf058: Preparing 
b9ab760d: Preparing 
cf2a0aad: Preparing 
72f12493: Preparing 
711a9638: Waiting g 
9e633e5: Pushed   1.441GB/1.394GB[11Alatest: digest: sha256:5b3036a236766aa61026356fe5eb194fc266432038b48183a9150ed7963919eb size: 3472


In [3]:
ecr_client = boto3.client('ecr')
response = ecr_client.describe_images(
    repositoryName=image_name,
    imageIds=[{'imageTag': 'latest'}],
)
str(response["imageDetails"][0]["imagePushedAt"])

'2022-04-04 14:56:03+00:00'

## Define parameters

In [6]:
opt_ml_dir = "/opt/ml/processing"
execution_id = "exp-real-sm"
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest"
train_data_uri = "s3://t5-engine-bucket/training_data/dummy/train/"
print(image_uri)
print(train_data_uri)

852039983533.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-t5:latest
s3://t5-engine-bucket/training_data/dummy/train/


In [7]:
processing_instance_count = 1
processing_instance_type = "ml.m5.2xlarge"
training_instance_type = "ml.m5.2xlarge"

## Train

In [8]:
import sagemaker
import json

# JSON encode hyperparameters
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

hyperparameters = json_encode_hyperparameters({"random_seed": 17})

est = sagemaker.estimator.Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type=training_instance_type,
    hyperparameters=hyperparameters,
    output_path=atf_s3_uri + f"/model",
    sagemaker_session=sagemaker_session,
    max_run=600, # timeout in seconds
    disable_profiler=True,
    use_spot_instances=True,
    max_wait=600, # <= max_run
)

est.fit({"train": train_data_uri})

2022-04-04 15:07:06 Starting - Starting the training job...
2022-04-04 15:07:08 Starting - Launching requested ML instances......
2022-04-04 15:08:36 Starting - Preparing the instances for training.........
2022-04-04 15:09:58 Downloading - Downloading input data
2022-04-04 15:09:58 Training - Downloading the training image......
2022-04-04 15:10:53 Training - Training image download completed. Training in progress.params: {'random_seed': '17'}
Start inspect_input
/opt/ml/input/data: ['train-manifest', 'train']
/opt/ml/input/data/train: ['train.csv']
Start train
Pycaret setup
IntProgress(value=0, description='Processing: ', max=3)
                                                                    
                                                                    
Initiated  . . . . . . . . . . . . . . . . . .              15:10:57
Status     . . . . . . . . . . . . . . . . . .  Loading Dependencies
                                                                           
         

### Inspect trained model artifact

In [9]:
job_name = est.latest_training_job.name
print(job_name)

training_job_description = est.jobs[-1].describe()
print(training_job_description['TrainingJobStatus'])
print(training_job_description['SecondaryStatus'])

model_data_s3_uri = f"{training_job_description['ModelArtifacts']['S3ModelArtifacts']}"
print(model_data_s3_uri)

sagemaker-t5-2022-04-04-15-07-06-524
Completed
Completed
s3://t5-engine-bucket/sagemaker/model/sagemaker-t5-2022-04-04-15-07-06-524/output/model.tar.gz


In [10]:
s3_helper.listdir(atf_s3_uri + f"/model/{job_name}/output")

[{'Key': 't5-engine-bucket/sagemaker/model/sagemaker-t5-2022-04-04-15-07-06-524/output/model.tar.gz',
  'LastModified': datetime.datetime(2022, 4, 4, 15, 11, 13, tzinfo=tzlocal()),
  'ETag': '"7955571d767f6115d2cc28435a066118"',
  'Size': 1353455,
  'StorageClass': 'STANDARD',
  'type': 'file',
  'size': 1353455,
  'name': 't5-engine-bucket/sagemaker/model/sagemaker-t5-2022-04-04-15-07-06-524/output/model.tar.gz'}]

In [11]:
# print out logs from Cloud Watch
logs = boto3.client("logs")

log_res = logs.describe_log_streams(
    logGroupName="/aws/sagemaker/TrainingJobs", logStreamNamePrefix=job_name
)

for log_stream in log_res["logStreams"]:
    # get one log event
    log_event = logs.get_log_events(
        logGroupName="/aws/sagemaker/TrainingJobs", logStreamName=log_stream["logStreamName"]
    )

    # print out messages from the log event
    for ev in log_event["events"]:
        for k, v in ev.items():
            if k == "message":
                print(v)

params: {'random_seed': '17'}
Start inspect_input
/opt/ml/input/data: ['train-manifest', 'train']
/opt/ml/input/data/train: ['train.csv']
Start train
Pycaret setup
IntProgress(value=0, description='Processing: ', max=3)
                                                                    
                                                                    
Initiated  . . . . . . . . . . . . . . . . . .              15:10:57
Status     . . . . . . . . . . . . . . . . . .  Loading Dependencies
                                                                           
                                                                           
Initiated  . . . . . . . . . . . . . . . . . .                     15:10:57
Status     . . . . . . . . . . . . . . . . . .  Preparing Data for Modeling
                                                                           
                                                                           
Initiated  . . . . . . . . . . . . . . . . . .  

## Evaluate

In [14]:
test_data_uri = "s3://t5-engine-bucket/training_data/dummy/test/"
print(test_data_uri)

s3://t5-engine-bucket/training_data/dummy/test/


In [15]:
print(opt_ml_dir)
evaluation_filename = "eval.json"
print(evaluation_filename)

/opt/ml/processing
eval.json


In [16]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

eval_processor = ScriptProcessor(
    base_job_name="evaluate-processor",
    image_uri=image_uri,
    command=['python'],
    role=role,
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
    max_runtime_in_seconds=1200,
)

eval_processor.run(
    code="container/code/evaluate.py",
    inputs=[
        ProcessingInput(
            source=model_data_s3_uri,
            destination=opt_ml_dir + "/model"
        ),
        ProcessingInput(
            source=test_data_uri,
            destination=opt_ml_dir + "/test"
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation",
            source=opt_ml_dir + "/evaluation",
            destination=atf_s3_uri + f"/evaluation/{execution_id}"
        ),
    ],
    wait=True,
    logs=True,
)


Job Name:  evaluate-processor-2022-04-04-15-12-15-129
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://t5-engine-bucket/sagemaker/model/sagemaker-t5-2022-04-04-15-07-06-524/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://t5-engine-bucket/training_data/dummy/test/', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-southeast-1-852039983533/evaluate-processor-2022-04-04-15-12-15-129/input/code/evaluate.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3Co

In [17]:
eval_job_description = eval_processor.jobs[-1].describe()
eval_output_config = eval_job_description["ProcessingOutputConfig"]
for output in eval_output_config["Outputs"]:
    if output["OutputName"] == "evaluation":
        eval_uri = output["S3Output"]["S3Uri"]
        
! aws s3 ls $eval_uri/

2022-04-04 15:16:09        173 eval.json


## Deploy

In [18]:
from sagemaker.predictor import CSVSerializer
predictor = est.deploy(
    initial_instance_count=1,
    instance_type=training_instance_type,
    serializer=CSVSerializer(),
)

----!

In [19]:
endpoint_name = predictor.endpoint_name
runtime = boto3.Session().client("runtime.sagemaker")
print(endpoint_name)
print(runtime)

sagemaker-t5-2022-04-04-15-16-28-438


## Test endpoint

In [20]:
import pandas as pd
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=default_bucket, Key=f'training_data/dummy/train/train.csv')
train_df = pd.read_csv(obj['Body'])
train_df

,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,150,target
0,-0.184958,-0.084,-0.354,0.355,-0.304,-0.058,0.880,11.46740,-0.049,442.693,...,471.1,-0.063,0.059045,11.109,-1.194949,-0.19373,-0.12037,0.064384,17.0970,0
1,0.662799,-0.243,-0.287,-0.741,2.560,-0.078,-0.263,9.45962,3.606,417.630,...,707.7,-0.035,0.050669,10.006,-1.015150,-0.17960,-0.10987,0.047429,17.3695,0
2,0.156325,-0.132,-0.235,0.752,-0.324,-0.438,-0.351,10.01040,-0.044,450.113,...,675.4,-0.012,0.041223,-21.501,1.274784,-0.18280,-0.11017,0.043130,17.1175,0
3,-0.661623,-0.178,-0.149,-0.923,-0.266,-0.184,2.681,9.45939,-0.059,455.255,...,580.7,-0.041,0.045162,-1.673,-1.193886,-0.18170,-0.12229,0.049549,17.5938,0
4,1.896510,-0.085,-0.359,0.764,-0.311,-0.078,-0.299,10.53900,-0.051,436.209,...,668.7,-0.033,0.055048,10.125,-1.205284,-0.19094,-0.11897,0.052379,17.5324,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.028558,-0.249,-0.288,-1.030,-0.334,-0.073,-0.320,11.34900,-0.072,443.974,...,375.9,-0.051,0.049026,1.273,-0.533203,-0.18568,-0.11735,0.053982,17.1961,3
296,-0.288804,-0.249,-0.288,-1.030,-0.334,-0.073,-0.320,12.03610,-0.072,450.603,...,389.9,-0.051,0.054744,-29.510,-0.725361,-0.21962,-0.11697,0.059700,16.9488,3
297,0.073675,-0.249,-0.288,-1.030,-0.334,-0.073,-0.320,10.44560,-0.072,427.511,...,625.4,-0.051,0.046761,13.455,-1.775220,-0.18374,-0.11664,0.045234,16.9493,1
298,-0.053419,-0.191,1.505,1.110,2.209,-0.062,-0.349,10.91210,-0.125,424.143,...,865.1,-0.015,0.054070,-15.422,1.265340,-0.18110,-0.12317,0.052165,16.9989,1


In [27]:
test_data = train_df.drop('target', axis=1)
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,-0.184958,-0.084,-0.354,0.355,-0.304,-0.058,0.880,11.46740,-0.049,442.693,...,1.058,471.1,-0.063,0.059045,11.109,-1.194949,-0.19373,-0.12037,0.064384,17.0970
1,0.662799,-0.243,-0.287,-0.741,2.560,-0.078,-0.263,9.45962,3.606,417.630,...,1.732,707.7,-0.035,0.050669,10.006,-1.015150,-0.17960,-0.10987,0.047429,17.3695
2,0.156325,-0.132,-0.235,0.752,-0.324,-0.438,-0.351,10.01040,-0.044,450.113,...,0.715,675.4,-0.012,0.041223,-21.501,1.274784,-0.18280,-0.11017,0.043130,17.1175
3,-0.661623,-0.178,-0.149,-0.923,-0.266,-0.184,2.681,9.45939,-0.059,455.255,...,0.406,580.7,-0.041,0.045162,-1.673,-1.193886,-0.18170,-0.12229,0.049549,17.5938
4,1.896510,-0.085,-0.359,0.764,-0.311,-0.078,-0.299,10.53900,-0.051,436.209,...,-1.051,668.7,-0.033,0.055048,10.125,-1.205284,-0.19094,-0.11897,0.052379,17.5324


### Test endpoint using predict function

In [28]:
def format_results(results):
    return list(map(float, results.split('\n')[:-1]))
results = predictor.predict(test_data.values).decode('utf-8')
format_results(results)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 4.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.0,
 3.0,
 3.0,
 3.0,
 3.0,
 1.0,
 2.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 3.0,
 2.0,
 2.0,
 3.0,
 3.0,
 3.0,
 3.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 1.0,
 2.0,
 2.0,
 1.0,
 4.0,
 4.0,
 1.0,
 4.0,
 2.0,
 4.0,
 2.0,
 1.0,
 2.0,
 3.0,
 3.0,
 3.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 2.0,
 2.0,
 3.0,
 0.0,
 2.0,
 1.0,
 2.0,
 1.0,
 0.0,
 2.0,
 4.0,
 1.0,
 1.0,
 2.0,
 1.0,
 4.0,
 2.0,
 4.0,
 3.0,
 4.0,
 0.0,
 4.0,
 2.0,
 1.0,
 2.0,
 4.0,
 2.0,
 4.0,
 1.0,
 2.0,
 2.0,
 2.0,
 1.0,
 3.0,
 3.0,
 4.0,
 1.0,
 2.0,
 1.0,
 2.0,
 2.0,
 4.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 4.0,
 1.0,
 1.0,
 4.0,
 2.0,
 1.0,
 1.0,
 1.0

### Test endpoint using invoke_endpoint function

In [29]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=CSVSerializer().serialize(test_data.values),
    ContentType='text/csv',
)
format_results(response['Body'].read().decode())

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 4.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.0,
 3.0,
 3.0,
 3.0,
 3.0,
 1.0,
 2.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 3.0,
 2.0,
 2.0,
 3.0,
 3.0,
 3.0,
 3.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 1.0,
 2.0,
 2.0,
 1.0,
 4.0,
 4.0,
 1.0,
 4.0,
 2.0,
 4.0,
 2.0,
 1.0,
 2.0,
 3.0,
 3.0,
 3.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 2.0,
 2.0,
 3.0,
 0.0,
 2.0,
 1.0,
 2.0,
 1.0,
 0.0,
 2.0,
 4.0,
 1.0,
 1.0,
 2.0,
 1.0,
 4.0,
 2.0,
 4.0,
 3.0,
 4.0,
 0.0,
 4.0,
 2.0,
 1.0,
 2.0,
 4.0,
 2.0,
 4.0,
 1.0,
 2.0,
 2.0,
 2.0,
 1.0,
 3.0,
 3.0,
 4.0,
 1.0,
 2.0,
 1.0,
 2.0,
 2.0,
 4.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 4.0,
 1.0,
 1.0,
 4.0,
 2.0,
 1.0,
 1.0,
 1.0

### Test endpoint using invoke_endpoint command line

In [30]:
test_data_str = CSVSerializer().serialize(test_data.values)
payload_file = "./payload"
with open(payload_file, "w") as f:
    f.write(test_data_str)
! aws sagemaker-runtime invoke-endpoint --endpoint-name $endpoint_name --body fileb://./payload --content-type text/csv outfile.txt && cat outfile.txt

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
4
1
0
0
0
0
0
0
0
0
0
4
3
3
3
3
1
2
1
2
1
1
1
1
1
3
2
2
3
3
3
3
1
1
2
2
2
1
2
2
1
4
4
1
4
2
4
2
1
2
3
3
3
1
2
2
1
1
2
2
1
1
2
2
3
0
2
1
2
1
0
2
4
1
1
2
1
4
2
4
3
4
0
4
2
1
2
4
2
4
1
2
2
2
1
3
3
4
1
2
1
2
2
4
1
1
1
1
1
1
1
4
1
1
4
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
3
3
3
3
4
2
4
1
4
2
2
3
3
3
3
2
4
4
3
3
3
3
1
1
1
1
1
2
2
2
1
2
1
4
2
4
1
2
4
1
1
1
1
2
1
3
3
1
3
1
1
4
1
2
2
3
3
3
3
2
1
3
3
3
3
2
1
2
1
1
2
1
1
1
1
1
2
2
4
1
4
1
3
3
1
3
2
2
3
3
1
3
3
1
4
2
2
1
2
2
1
3
3
3
3
1
1
0


In [31]:
# print out logs from Cloud Watch
logs = boto3.client("logs")

log_res = logs.describe_log_streams(
    logGroupName=f"/aws/sagemaker/Endpoints/{endpoint_name}"
)

for log_stream in log_res["logStreams"]:
    # get one log event
    log_event = logs.get_log_events(
        logGroupName=f"/aws/sagemaker/Endpoints/{endpoint_name}", logStreamName=log_stream["logStreamName"]
    )

    # print out messages from the log event
    for ev in log_event["events"]:
        for k, v in ev.items():
            if k == "message":
                print(v)

Starting the inference server with 8 workers.
[2022-04-04 15:17:36 +0000] [10] [INFO] Starting gunicorn 20.1.0
[2022-04-04 15:17:36 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2022-04-04 15:17:36 +0000] [10] [INFO] Using worker: sync
[2022-04-04 15:17:36 +0000] [13] [INFO] Booting worker with pid: 13
[2022-04-04 15:17:36 +0000] [14] [INFO] Booting worker with pid: 14
[2022-04-04 15:17:36 +0000] [22] [INFO] Booting worker with pid: 22
[2022-04-04 15:17:36 +0000] [23] [INFO] Booting worker with pid: 23
[2022-04-04 15:17:36 +0000] [24] [INFO] Booting worker with pid: 24
[2022-04-04 15:17:36 +0000] [32] [INFO] Booting worker with pid: 32
[2022-04-04 15:17:36 +0000] [33] [INFO] Booting worker with pid: 33
[2022-04-04 15:17:36 +0000] [42] [INFO] Booting worker with pid: 42
Pycaret load_config
Pycaret load_config
Pycaret load_config
Pycaret load_model
Transformation Pipeline and Model Successfully Loaded
Pycaret load_config
Pycaret load_config
169.254.178.2 - - [04/Apr/2022

In [32]:
predictor.delete_endpoint()

In [33]:
! rm payload && rm outfile.txt